In [15]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [16]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [17]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Trousers & Jeans - Female - Test.npz')
data_validation = np.load(r'Full Dataset/Trousers & Jeans - Female - Validation.npz')
data_test = np.load(r'Full Dataset/Trousers & Jeans - Female - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [18]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))


METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/ModelF 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [19]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(64, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(64, activation = 'relu'),
                
                Dense(4)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/ModelF 1/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(6, 6))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Trouser", "Jeans"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"saved_model/ModelF 1/fit/run-{session_num}")


    return accuracy



In [20]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/ModelF 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
8/8 [==============================] - 0s 32ms/step
4/4 - 3s - loss: 1.3358 - accuracy: 0.5280 - val_loss: 0.6796 - val_accuracy: 0.5760 - 3s/epoch - 850ms/step
Epoch 2/15
8/8 [==============================] - 0s 32ms/step
4/4 - 2s - loss: 0.8861 - accuracy: 0.4920 - val_loss: 0.6923 - val_accuracy: 0.5160 - 2s/epoch - 478ms/step
Epoch 3/15
8/8 [==============================] - 0s 30ms/step
4/4 - 2s - loss: 0.6749 - accuracy: 0.6040 - val_loss: 0.6668 - val_accuracy: 0.6800 - 2s/epoch - 481ms/step
Epoch 4/15
8/8 [==============================] - 0s 33ms/step
4/4 - 2s - loss: 0.6215 - accuracy: 0.6760 - val_loss: 0.7313 - val_accuracy: 0.6000 - 2s/epoch - 471ms/step
Epoch 5/15
8/8 [==============================] - 0s 36ms/step
4/4 - 2s - loss: 0.6309 - accuracy: 0.6320 - val_loss: 0.6094 - val_accuracy: 0.7120 - 2s/epoch - 488ms/step
Epoch 6/15
8/8 [==============================] - 0s 34ms/step
4/4 - 2s - loss

INFO:tensorflow:Assets written to: saved_model/ModelF 1/fit/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}
Epoch 1/15
8/8 [==============================] - 0s 38ms/step
4/4 - 3s - loss: 2.3736 - accuracy: 0.4480 - val_loss: 3.1099 - val_accuracy: 0.5160 - 3s/epoch - 819ms/step
Epoch 2/15
8/8 [==============================] - 0s 31ms/step
4/4 - 2s - loss: 2.4473 - accuracy: 0.5080 - val_loss: 0.9606 - val_accuracy: 0.5200 - 2s/epoch - 505ms/step
Epoch 3/15
8/8 [==============================] - 0s 34ms/step
4/4 - 2s - loss: 1.0042 - accuracy: 0.5320 - val_loss: 0.6892 - val_accuracy: 0.6600 - 2s/epoch - 489ms/step
Epoch 4/15
8/8 [==============================] - 0s 31ms/step
4/4 - 2s - loss: 0.7287 - accuracy: 0.6560 - val_loss: 0.7736 - val_accuracy: 0.7000 - 2s/epoch - 472ms/step
Epoch 5/15
8/8 [==============================] - 0s 34ms/step
4/4 - 2s - loss: 0.6499 - accuracy: 0.7280 - val_loss: 0.6118 - val_accuracy: 0.6680 - 2s/epoch - 501ms/step
Epoch 6/15
8/8 [==============================] - 0s 40ms/step
4/4 - 2s - loss

INFO:tensorflow:Assets written to: saved_model/ModelF 1/fit/run-2\assets


---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}
Epoch 1/15
8/8 [==============================] - 0s 34ms/step
4/4 - 3s - loss: 1.1101 - accuracy: 0.3880 - val_loss: 0.7967 - val_accuracy: 0.4840 - 3s/epoch - 756ms/step
Epoch 2/15
8/8 [==============================] - 0s 33ms/step
4/4 - 2s - loss: 0.7344 - accuracy: 0.5520 - val_loss: 0.6704 - val_accuracy: 0.5240 - 2s/epoch - 532ms/step
Epoch 3/15
8/8 [==============================] - 0s 37ms/step
4/4 - 2s - loss: 0.6546 - accuracy: 0.5720 - val_loss: 0.6273 - val_accuracy: 0.7320 - 2s/epoch - 555ms/step
Epoch 4/15
8/8 [==============================] - 0s 33ms/step
4/4 - 2s - loss: 0.5770 - accuracy: 0.7360 - val_loss: 0.5824 - val_accuracy: 0.7080 - 2s/epoch - 494ms/step
Epoch 5/15
8/8 [==============================] - 0s 31ms/step
4/4 - 2s - loss: 0.5262 - accuracy: 0.7560 - val_loss: 0.6282 - val_accuracy: 0.6760 - 2s/epoch - 513ms/step
Epoch 6/15
8/8 [==============================] - 0s 40ms/step
4/4 - 2s - loss

...

INFO:tensorflow:Assets written to: saved_model/ModelF 1/fit/run-10\assets


---Starting trial: run-11
{'filter_size': 7, 'filter_num': 96}
Epoch 1/15
8/8 [==============================] - 1s 62ms/step
4/4 - 6s - loss: 1.3431 - accuracy: 0.4040 - val_loss: 0.7062 - val_accuracy: 0.5160 - 6s/epoch - 1s/step
Epoch 2/15
8/8 [==============================] - 1s 65ms/step
4/4 - 4s - loss: 0.7198 - accuracy: 0.5760 - val_loss: 0.7113 - val_accuracy: 0.6000 - 4s/epoch - 939ms/step
Epoch 3/15
8/8 [==============================] - 1s 67ms/step
4/4 - 4s - loss: 0.6189 - accuracy: 0.6920 - val_loss: 0.6107 - val_accuracy: 0.6880 - 4s/epoch - 988ms/step
Epoch 4/15
8/8 [==============================] - 1s 69ms/step
4/4 - 4s - loss: 0.5412 - accuracy: 0.7280 - val_loss: 0.6219 - val_accuracy: 0.6920 - 4s/epoch - 919ms/step
Epoch 5/15
8/8 [==============================] - 1s 64ms/step
4/4 - 4s - loss: 0.4791 - accuracy: 0.7760 - val_loss: 0.6107 - val_accuracy: 0.7040 - 4s/epoch - 929ms/step
8/8 [==============================] - 1s 63ms/step - loss: 0.6107 - accuracy: 0

INFO:tensorflow:Assets written to: saved_model/ModelF 1/fit/run-11\assets


---Starting trial: run-12
{'filter_size': 7, 'filter_num': 128}
Epoch 1/15
8/8 [==============================] - 1s 66ms/step
4/4 - 6s - loss: 1.7711 - accuracy: 0.3600 - val_loss: 0.6837 - val_accuracy: 0.5200 - 6s/epoch - 2s/step
Epoch 2/15
8/8 [==============================] - 1s 65ms/step
4/4 - 4s - loss: 1.0238 - accuracy: 0.5080 - val_loss: 0.7018 - val_accuracy: 0.6320 - 4s/epoch - 904ms/step
Epoch 3/15
8/8 [==============================] - 1s 69ms/step
4/4 - 4s - loss: 0.7420 - accuracy: 0.5560 - val_loss: 0.7516 - val_accuracy: 0.5160 - 4s/epoch - 950ms/step
8/8 [==============================] - 1s 69ms/step - loss: 0.6837 - accuracy: 0.5200
INFO:tensorflow:Assets written to: saved_model/ModelF 1/fit/run-12\assets


INFO:tensorflow:Assets written to: saved_model/ModelF 1/fit/run-12\assets


In [21]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelF 1/hparam_tuning'


In [22]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/ModelF 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
